In [ ]:
import argparse
import os, sys
import torch
import numpy as np
from tqdm import tqdm
import datetime, random
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from datasets import Ambisonizer
from model.seanet import SEANet
from utils import seed_worker, set_seed
import auraloss

device = 'cpu'

checkpoint_path = None # Fill this in with the path to the checkpoint you want to load

model = SEANet(480000, 64).to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
print(model)

In [ ]:
source_audio = None # Fill this in with the path to the audio file you want to use
import librosa
y, sr = librosa.load(source_audio, sr=44100, mono=False)
start_idx = np.random.randint(0, y.shape[1] - 480000)
y = y[:, start_idx:start_idx+480000]
print(y.shape)

In [ ]:
y = torch.tensor(y).to(device).float().unsqueeze(0)
print(y.shape)
y_pred, _, _ = model(y)
print(y_pred.shape)

In [ ]:
# save y
y = y.squeeze().detach().cpu().numpy()
y = np.transpose(y, (1, 0))
y = np.ascontiguousarray(y)
import soundfile as sf
sf.write('y.wav', y, 44100)
y_mono = np.mean(y, axis=1)
sf.write('y_mono.wav', y_mono, 44100)

In [ ]:
y_pred = y_pred.squeeze().detach().cpu().numpy()
x = y_pred[0]
y = y_pred[1]
w = np.mean(y, axis=0)

sf.write('X.wav', x, 44100)
sf.write('Y.wav', y, 44100)
sf.write('W.wav', w, 44100)